# The Die Rolling Problem (Python3)

In [70]:
#code code code
import math
import random

In [3]:
print("answers")

answers


In [ ]:
# "fbit" is short for "fractional bit," so called because Bernoulli r.v.s often have less than one bit of entropy.

In [ ]:
# Information Pool
information_pool = []

In [ ]:
# 'probability' is the probability (known beforehand) that the Bernoulli r.v. is True.
# 'result' is the value of the Bernoulli r.v., either True or False.
def store_information(probability, result):
    information_pool = [(probability, result)] + entropy_pool

In [30]:
# We need an actual source of entropy, in the form of (assumed) random bits.
# The actual source of entropy is a generator function, that returns one random bit at a time.
# For the purposes of analysis, it comes bundled in a dictionary with an integer that keeps track
# of the number of bits accessed so far.

# An entropy source is a (bit_generator, num_bits_consumed) pair.
def make_pseudorandom_entropy_source():
    result = {
        "num_bits_consumed": 0
    }
    
    def make_pseudorandom_bit_generator():
        while True:
            result["num_bits_consumed"] += 1
            yield True if random.getrandbits(1) else False
    
    result["bit_generator"] = make_pseudorandom_bit_generator()
    return result

In [55]:
# Test out the entropy source.
zoop = make_pseudorandom_entropy_source()
for i in range(10):
    print(next(zoop["bit_generator"]))
print(zoop["num_bits_consumed"], "bits consumed so far.")

False
False
False
True
False
False
False
True
True
False
10 bits consumed so far.


In [147]:
# Figure out how many bits are necessary to represent at least n different values.
# Basically a base two logarithm.
def bit_span(n):
    bits_required = 0
    test_bit = 1
    while test_bit < n:
        bits_required += 1
        test_bit <<= 1
    return bits_required

# Gather enough random bits such that the bits, when treated as an unsigned integer,
# could be at least as large as n - 1, so that it covers the whole range [0, n).
# Use a redundancy to generate at least 'redundancy' times as many bits.
def gather_bits(entropy_source, n, redundancy=1.0):
    num_bits = math.ceil(bit_span(n) * redundancy)
    bits = 0
    positioned_bit = 1
    for bit_index in range(num_bits):
        if next(entropy_source["bit_generator"]):
            bits |= positioned_bit
        positioned_bit <<= 1
    return bits

In [149]:
# Test it out
foo = make_pseudorandom_entropy_source()
print(gather_bits(foo, 123))
print(foo["num_bits_consumed"], "bits consumed so far.")

124
7 bits consumed so far.


In [110]:
# Next, we will implement a variety of different die-rolling algorithms
# and compare their entropy usage, both in theory and by running some experiments.

In [104]:
# Gather the minimum number of bits required, then perform a modulo
# on the random bits and return the result.
def modulo_die_roller(entropy_source, n):
    return gather_bits(entropy_source, n) % n

In [105]:
# Test it out
foo = make_pseudorandom_entropy_source()
print(modulo_die_roller(foo, 123)) # 7 bits required
print(foo["num_bits_consumed"], "bits consumed so far.")

108
7 bits consumed so far.


In [ ]:
# TODO: use matplotlib to plot the modulo function, to show that it's not fair.

In [ ]:
# Same as above, but use extra bits to improve the fairness.
def generous_modulo_die_roller(entropy_source, n, redundancy):
    assert redundancy >= 1.0

In [106]:
# TODO: plot redundancy versus estimated fairness with lots of data

In [150]:
def rejection_sampling_die_roller(entropy_source, n):
    while True:
        random_bits = gather_bits(entropy_source, n)
        if random_bits < n:
            return random_bits

In [151]:
def generous_rejection_sampling_die_roller(entropy_source, n, redundancy):
    num_bits = bit_span(n)
    # There are exactly n * M values <= n, where M is a positive integer.
    max_accepted_value = ((1 << bits_to_use) // n) * n - 1
    while True:
        random_bits = gather_bits(entropy_source, n, redundancy)
        if random_bits <= max_accepted_value:
            return random_bits

In [152]:
# TODO: graphs

In [ ]:
# TODO: look back at methods so far, summarize pros and cons
# recognize tradeoff between fairness, runtime, and entropy usage.


In [ ]:
# TODO: more entropy-efficient methods

In [1]:
def range_encode(sequence, limits):
    assert type(sequence) is list or type(sequence) is iter
    assert type(limits) is list or type(limits) is iter
    assert len(sequence) <= len(limits)
    
    if type(sequence) is list:
        sequence = iter(sequence)
    
    if type(limits) is list:
        limits = iter(limits)
    
    encoded = 0
    factor = 1
    for value in sequence:
        next_limit = next(limits)
        if value >= next_limit:
            raise Exception("Error: each value[n] must be in range(limit[n])")
        
        encoded += value * factor
        factor *= next_limit
    
    return encoded

In [2]:
def range_decode(encoded, limits):
    assert type(limits) is list or type(limits) is iter
    
    result = []
    for limit in limits:
        result.append(encoded % limit)
        encoded //= limit

    return result